# WaveSolver - 1D
M. Lamoureux May 31, 2016. Pacific Insitute for the Mathematical Sciences. Updated June 15, 2016.

## Demo notes.
Run "All Cells" then look at the last graph at the bottom. The slides will control various numerical solutions to the wave equation. So if you don't want to read all the section, you can just see the end results at the bottom.

You can also examine each section individual, if you want to understand the numeric behind solving the one-dimensional wave equation (with constant coefficients). Becuase of a quirk in Boken, you have to run the cell with the plot, followed by the cell with the slider, in order to get each plot to work individually. 

This was also written as a test of how far we can push Bokeh and the Jupyter Hub in making some calculations and graphics. 

## Math notes
We have three examples of a numerical solver for the one dimensional wave equation: 
- explicit finite difference
- implicit finite difference
- D'Alembert's exact solution. 

In the demo, we show solutions for an initial position of  a triangular bump, then for an initial particle velocity of a triangular bump, and finally we compare the exact solution (d'Alembert) to the finite difference code. For fun, we also show how a moving Gaussian propagates. 




## Introduction to the 1D wave equation.

The basic idea is to represent the vibrations of a (horizonal) string under tension by a function $u(x,t)$ where $x$ is the horizontal displacement along the string, $t$ is the time parameter, and $y=u(x,t)$ indicates the (vertical) displacement of the string at point $x$ along the string, at time $t$ 

A typical wave would look something like
$$u(x,t) = \sin(x-ct)$$
which represents a sinusoidal wave moving to the right at velocity $c$.

From Newon's law of motion (mass times acceleration equals force), we can derive the wave equation for the string in the form 
$$ \rho \cdot u_{tt} = k \cdot u_{xx}$$
where $\rho$ is a mass density, $u_{tt}$ is an acceleration (the second derivative w.r.t. time), $k$ is the modulus of elasticity and $u_{xx}$ is a measure of curvature (the second derivative w.r.t. $x$), causing the bending force. 

Including boundary and initial conditions, while setting the parameter $c^2 = k/\rho$, we obtain the usual 1D wave equation on an interval as this:


$$u_{tt} = c^2 u_{xx} \mbox{ on interval } [x_0,x_1]$$
subject to boundary conditions
$$u(x_0,t) = u(x_1, t) = 0 \mbox{ for all $t$. }$$
and initial conditions
$$u(x,0) = u_0(x), \quad u_t(x,t) = u_1(x) \mbox{ on interval } [x_0,x_1].$$


Let's write some PDE solvers for this 1D wave equation.

Start with some simple solvers, then get more complex. 

The function to solve the PDE needs to know $c^2, u_0, u_1$, the sampling intervals $dx$ and $dt$, and the number of time steps to execute. Everything else can be inferred from those inputs. The output should be a 2D array, indexed in x and t steps. 

Maybe we could actually output the solution $u(x,t)$ as well as the vector of indices for $x$ and $t$.

## Software tools
We import some tools for numerical work (NumPy, SciPy), plotting (Boken), and an interactive widget. We are using Boken because it is faster than matplotlib. Note, however, that Boken seems to be heavy on memory usage. 

You can read about the interactive widgets here: http://bokeh.pydata.org/en/0.10.0/_images/notebook_interactors.png

In [1]:
import numpy as np
from scipy.linalg import solve_toeplitz  # a matrix equation solver
from scipy.integrate import cumtrapz # a numerical integrator, using trapezoid rule
from bokeh.models import ColumnDataSource
from bokeh.plotting import *
from bokeh.io import push_notebook
from ipywidgets import interact, IntSlider

We need to initialize the Bokeh plotting methods with the following. 

In [2]:
output_notebook()

Loading BokehJS ...

## Explicit method of solution in finite differences

The finite difference method is probably the most common numerical method for solving PDEs. The derivatives are approximated by Newton difference ratios, and you step through in time to progress the solution from $t=0$ to some ending time.

The following defines a function that solves the wave equation, using an explicit finite difference method. We follow the notes in the referenced book by Myint-U and Debnath. 

The differencing in spatial variable $x$ is done by a convolution, as this will be fast. 

We print out the value of the CFL constant as a sanity check. The method is stable and convergent provided the CFL value is less than one. (This means $dt$ needs to be small enough.) See the book for details. 

The input parameters are velocity squared $c2$, spatial step size $dx$, temporal step size $dt$, number of time steps $t$_$len$, initial position $u0$ and initial velocity $u1$. $u0$ and $u1$ are arrays of some length $N$, which the code will use to deduce everything it needs to know.

I tend to think of these as dimensionless variables, but you can use real physical values if you like, so long as you are consistent. For instance $dx$ in meters, $dt$ in seconds, and $c2$ in (meters/second) squared. 

In [3]:
# Based on Section 14.4 in Myint-U and Debnath's book
def w_solve1(c2,dx,dt,t_len,u0,u1):
    x_len = np.size(u0)  # the length of u0 implicitly defines the num of points in x direction
    u = np.zeros((x_len,t_len),order='F') # output array initialized to zero 
    e2 = c2*dt*dt/(dx*dx)  # Courant parameter squared (test for convergence!)
    print("CFL value is ",np.sqrt(e2))
    kern = np.array([e2, 2*(1-e2), e2]) # the convolution kernel we need for laplacian solver
    u[:,0] = u0            # t=0 initial condition
    u[:,1] = np.convolve(u0,kern/2)[1:x_len+1] + dt*u1 # t=0 derivative condition, eqn 14.4.6
    for j in range(2,t_len):
        u[:,j] = np.convolve(u[:,j-1],kern)[1:x_len+1] - u[:,j-2] # eqn 14.4.3
    # let's produce the x and t vectors, in case we need them
    x = np.linspace(0,dx*x_len,x_len,endpoint=False)
    t = np.linspace(0,dt*t_len,t_len,endpoint=False)
    return u,x,t

### First example
We do 1000 steps in time and space, using a triangular waveform for the initial values. We then call the wave equation solver. Finally, we set up Bokeh to plot the answers for us. 

In [27]:
# Define the initial parameters, including a triangular waveform
c2 = .5  # velocity squared
x_len = 1000
t_len = 1000
dx = 1./x_len
dt = 1./t_len
x = np.linspace(0,1,x_len)
t = np.linspace(0,1,t_len)
triangle = np.maximum(0.,.1-np.absolute(x-.4))

# Here we solve the wave equation, with initial position $u(x,0)$ set to the triangle waveform
(u_exp,x,t)=w_solve1(c2,dx,dt,t_len,triangle,0*triangle)

# And now we plot the result, using Bokeh as our plotting method

y=u_exp[:,0]
src = ColumnDataSource(data=dict(x=x,y=y))

p = figure(title="FD Explicit method solution", plot_height=300, plot_width=600)
p.line(x,y,color="#2222aa",line_width=3,name="foo")

def update_exp(k=0):
    src.data['y']=u_exp[:,k]
    p.title='FD Explicit method solution"
    push_notebook()

show(p)

CFL value is  0.707106781187


In [28]:
# We have to call up the slider in a separate cell
interact(update_exp,k=(0,t_len-1))

<function __main__.update_exp>

### Second example
As above, but we use the triangular waveform for the initial particle velocities. We take 3000 time steps, because the veolocity initial condition is a bit unusual, and we want to really see what is going on. 

In [6]:
# Define the initial parameters, including a triangular waveform
c2 = .5  # velocity squared
x_len = 1000
t_len = 1000
dx = 1./x_len
dt = 1./t_len
x = np.linspace(0,1,x_len)
t = np.linspace(0,1,t_len)
triangle = np.maximum(0.,.1-np.absolute(x-.4))

# We solve, but this time with the velocity $u_t$ initial condition equal to the triangle impulse. 3000 steps!
(u_expD,x,t)=w_solve1(.5,dx,dt,3*t_len,0*triangle,1*triangle)

# And now we plot the result, using Bokeh as our plotting method

y=u_expD[:,0]
src = ColumnDataSource(data=dict(x=x,y=y))

p = figure(title="F.D. Explicit method, init velocity", plot_height=300, plot_width=600)
p.line(x,y,color="#2222aa",line_width=3,source=src,name="foo")

def update_expD(k=0):
    src.data['y']=u_expD[:,k]
    p.title="FD Explicit method, init velocity"
    push_notebook()

show(p)

CFL value is  0.707106781187


/Users/mikel/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/mikel/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


In [7]:
# We have to call up the slider in a separate cell
interact(update_expD,k=(0,3*t_len-1))

<function __main__.update_expD>

## Implicit method

Here we try an implicit method for solving the wave equation. Again from Myint-U and Debnath's book, in Section 14.5, part (B) on Hyperbolic equations. 

We need to use scipy libraries as we have to solve a system of linear equation. In fact the system is tridiagonal and Toepliz, so this should be fast. I see how to use Toepliz in scipy, but I don't know how to tell it that the system is only tridiagonal. It should be possible to speed this up. 

In [8]:
# Based on Section 14.5 (B) in Myint-U and Debnath's book
def w_solve2(c2,dx,dt,t_len,u0,u1):
    x_len = np.size(u0)  # the length of u0 implicitly defines the num of points in x direction
    u = np.zeros((x_len,t_len),order='F') # output array initialized to zero 
    e2 = c2*dt*dt/(dx*dx)  # Courant parameter squared (test for convergence!)
    print("CFL value is ",np.sqrt(e2))
    kern = np.array([e2, 2*(1-e2), e2]) # the convolution kernel we need for laplacian solver
    u[:,0] = u0            # t=0 initial condition
    u[:,1] = np.convolve(u0,kern/2)[1:x_len+1] + dt*u1 # t=0 derivative condition, eqn 14.4.6
    # Note the above is a cheat, we are using the explicit method to find u[:,1], Should do this implicitly
    kern2 = np.array([e2, -2*(1+e2), e2]) # the convolution kernel we need for implicit solver. It is different.
    toepk = np.zeros(x_len)  # this will hold the entries for the tridiagonal Toeplitz matrix
    toepk[0] = 2*(1+e2);
    toepk[1] = -e2
    for j in range(2,t_len):
        rhs = np.convolve(u[:,j-2],kern2)[1:x_len+1] + 4*u[:,j-1] # eqn 14.5.17
        u[:,j] = solve_toeplitz(toepk, rhs) # here is a linear system solver (hence an implicit method)
    # let's produce the x and t vectors, in case we need them
    x = np.linspace(0,dx*x_len,x_len,endpoint=False)
    t = np.linspace(0,dt*t_len,t_len,endpoint=False)
    return u,x,t

### Third example
We do 1000 steps in time and space, using a triangular waveform for the initial values. We then call the implicit wave equation solver, which is supposed to be better. Finally, we set up Bokeh to plot the answers for us. 

In [9]:
# Define the initial parameters, including a triangular waveform
c2 = .5  # velocity squared
x_len = 1000
t_len = 1000
dx = 1./x_len
dt = 1./t_len
x = np.linspace(0,1,x_len)
t = np.linspace(0,1,t_len)
triangle = np.maximum(0.,.1-np.absolute(x-.4))

# solve with the implicit method
(u_imp,x,t)=w_solve2(c2,dx,dt,t_len,1*triangle,0*triangle)

# And now we plot the result

y=u_imp[:,0]
src = ColumnDataSource(data=dict(x=x,y=y))

p = figure(title="FD Implicit method solution", plot_height=300, plot_width=600)
p.line(x,y,color="#2222aa",line_width=3,source=src,name="foo")

def update_imp(k=0):
    src.data['y']=u_imp[:,k]
    p.title="FD Implicit method solution"
    push_notebook()

show(p)

CFL value is  0.707106781187


/Users/mikel/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/mikel/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


In [10]:
interact(update_imp,k=(0,t_len-1))

<function __main__.update_imp>

### Fourth example
Derivative initial condition. Using the implicit solver, we solve for the initial particle velocity as a triangular waveform. 

In [11]:
# Define the initial parameters, including a triangular waveform
c2 = .5  # velocity squared
x_len = 1000
t_len = 1000
dx = 1./x_len
dt = 1./t_len
x = np.linspace(0,1,x_len)
t = np.linspace(0,1,t_len)
triangle = np.maximum(0.,.1-np.absolute(x-.4))

# Solve using the implicit solver. We do 3000 time steps to see what is going on. 
(u_impD,x,t)=w_solve2(c2,dx,dt,3*t_len,0*triangle,1*triangle)

# And now we plot the result

y=u_impD[:,0]
src = ColumnDataSource(data=dict(x=x,y=y))

p = figure(title="FD Implicit method, init velocity", plot_height=300, plot_width=600)
p.line(x,y,color="#2222aa",line_width=3,source=src,name="foo")

def update_impD(k=0):
    src.data['y']=u_impD[:,k]
    p.title="FD Implicit method, init velocity"
    push_notebook()

show(p)

CFL value is  0.707106781187


/Users/mikel/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/mikel/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


In [12]:
interact(update_impD,k=(0,3*t_len-1))

<function __main__.update_impD>

## D'Alembert's solution

Since the velocity $c$ is constant in these examples, we can get the exact solution via D'Alembert. The general solution will be of the form 
$$u(x,t) = \phi(x+ct) + \psi(x-ct). $$
Initial conditions tell use that
$$u(x,0) = \phi(x) + \psi(x) = f(x), $$ and
$$u_t(x,0) = c\phi'(x) - c\psi'(x) = g(x). $$
With $G(x)$ the antiderivative of $g(x)$ with appropriate zero at zero, we get a 2x2 system
$$\phi(x) + \psi(x) = f(x), \\ c(\phi(x) - \psi(x)) = G(x),$$
which we solve as
$$\phi(x) = \frac{1}{2}\left( f(x) + \frac{1}{c} G(x) \right), \\
  \psi(x) = \frac{1}{2}\left( f(x) - \frac{1}{c} G(x) \right).$$

Now $f(x)$ is given as the argument $u0$ in the code. $G(x)$ can be computed using scipy. The arguments $x+ct$ and $x-ct$ must be converted to integer indices. They have to wrap around. And with the zero boundary condition, we need to wrap around with a negative reflection. 

There is the messy question as to whether we require $u(0,t)$ to actually equal zero, or do we require it to be zero one index "to the left" of x=0. Let's not think too much about that just yet.

In [13]:
# Based on D'Alembert's solution, as described above
def w_solve3(c2,dx,dt,t_len,u0,u1):
    x_len = np.size(u0)  # the length of u0 implicitly defines the num of points in x direction
    u = np.zeros((x_len,t_len),order='F') # output array initialized to zero 
    c = np.sqrt(c2)  # the actual velocity parameter is needed
    f = u0  # use notation from above notes
    G = cumtrapz(u1,dx=dx,initial=0) # the antiderivative, using cumulative trapezoidal rule
    f2 = np.append(f,-f[::-1]) # odd symmetry
    G2 = np.append(G,G[::-1]) # even symmetry
    phi2 = (f2 + G2/c)/2
    psi2 = (f2 - G2/c)/2
    x = np.linspace(0,dx*x_len,x_len,endpoint=False)
    t = np.linspace(0,dt*t_len,t_len,endpoint=False)
    # in the loop, we convert x+ct to index's into vectors phi2, psi2, modulo the vector length
    for j in range(t_len):
        ii1 = np.mod( np.round((x+c*t[j])/dx), 2*x_len)
        ii2 = np.mod( np.round((x-c*t[j])/dx), 2*x_len)
        u[:,j] = phi2[ii1.astype(int)] + psi2[ii2.astype(int)]
    return u,x,t

### Fifth example

We do 1000 steps in time and space, using a triangular waveform for the initial values. 
We then call the d'Alembert wave equation solver, which should be exact to machine precision. 
Finally, we set up Bokeh to plot the answers for us.

In [14]:
# Define the initial parameters, including a triangular waveform
c2 = .5  # velocity squared
x_len = 1000
t_len = 1000
dx = 1./x_len
dt = 1./t_len
x = np.linspace(0,1,x_len)
t = np.linspace(0,1,t_len)
triangle = np.maximum(0.,.1-np.absolute(x-.4))

# solve using our d'Alembert solver
(u_dal,x,t)=w_solve3(c2,dx,dt,t_len,1*triangle,0*triangle)

# and now we plot

y=u_dal[:,0]
src = ColumnDataSource(data=dict(x=x,y=y))

p = figure(title="D'Alembert solution", plot_height=300, plot_width=600)
p.line(x,y,color="#2222aa",line_width=3,source=src,name="foo")

def update_dal(k=0):
    src.data['y']=u_dal[:,k]
    p.title="D'Alembert solution"
    push_notebook()

show(p)

/Users/mikel/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/mikel/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


In [15]:
interact(update_dal,k=(0,t_len-1))


<function __main__.update_dal>

### Sixth example
Derivative initial condition. Using the exact solver, we solve for the initial particle velocity as a triangular waveform. Derivative initial condition.

In [16]:
# Define the initial parameters, including a triangular waveform
c2 = .5  # velocity squared
x_len = 1000
t_len = 1000
dx = 1./x_len
dt = 1./t_len
x = np.linspace(0,1,x_len)
t = np.linspace(0,1,t_len)
triangle = np.maximum(0.,.1-np.absolute(x-.4))

# solve using our d'Alembert solver. We do 3000 time steps, to see what is going on.
(u_dalD,x,t)=w_solve3(c2,dx,dt,3*t_len,0*triangle,1*triangle)

# And now we plot the result

y=u_dalD[:,0]
src = ColumnDataSource(data=dict(x=x,y=y))

p = figure(title="D'Alembert method, init velocity", plot_height=300, plot_width=600)
p.line(x,y,color="#2222aa",line_width=3,source=src,name="foo")

def update_dalD(k=0):
    src.data['y']=u_dalD[:,k]
    p.title="D'Alembert method, init velocity"
    push_notebook()

show(p)

/Users/mikel/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/mikel/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


In [17]:
interact(update_dalD,k=(0,3*t_len-1))

<function __main__.update_dalD>

# Comparing solutions

### Seventh example

In principle, we want these different solution methods to be directly comparable.

So let's try this out, by computing the difference of two solution. 

Here we compare the explicit f.d. method with d'Alambert's method.


In [18]:
# If you want, you can re-compute these. But we did it above already.
#(u_exp,x,t)=w_solve1(.5,dx,dt,t_len,1*triangle,0*triangle)
#(u_dal,x,t)=w_solve3(.5,dx,dt,t_len,1*triangle,0*triangle)

y=u_dal[:,0]-u_exp[:,0]
src = ColumnDataSource(data=dict(x=x,y=y))

p = figure(title="D'Alembert minus explicit", plot_height=300, plot_width=600)
p.line(x,y,color="#2222aa",line_width=3,source=src,name="foo")

def update_comp(k=0):
    src.data['y']=u_dal[:,k]-u_exp[:,k]
    p.title="D'Alembert minus explicit"
    push_notebook()
    
show(p)

/Users/mikel/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/mikel/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


In [19]:
interact(update_comp,k=(0,t_len-1))

<function __main__.update_comp>

# A moving wavefront

### Example eight

Let's try an actual wave. We want something like
$$u(x,t) = \exp(-(x -x_a-ct)^2/w^2), $$
where $x_a$ is the center of the Gaussian at $t=0$, $w$ is the width of the Gaussian, $c$ is the velocity of the wave.

This gives
$$u_0(x) = \exp(-(x -x_a)^2/w^2) \\
  u_1(x) = \frac{2c(x-x_a)}{w^2}\exp(-(x -x_a)^2/w^2) = \frac{2c(x-x_a)}{w^2}u_0(x).$$


In [20]:
# We set up the initial waveform corresponding to a Gaussian moving to the right.
# The value of velocity c is very important here!
c = .707  # velocity
x_len = 1000
t_len = 1000
dx = 1./x_len
dt = 1./t_len
x = np.linspace(0,1,x_len)
t = np.linspace(0,1,t_len)
u0 = np.exp(-(x-.5)*(x-.5)/.01)
u1 = 2*c*u0*(x-.5)/.01

# now we solve using the d'Alembert solution. We do 3000 steps, though!

(u_wav,x,t)=w_solve3(c*c,dx,dt,3*t_len,u0,u1)  # notice we input the velocity squared!

# And now we plot

y=u_wav[:,0]
src = ColumnDataSource(data=dict(x=x,y=y))
p = figure(title="Moving wave", plot_height=300, plot_width=600)
p.line(x,y,color="#2222aa",line_width=3,source=src,name="foo")

def update_wav(k=0):
    src.data['y']=u_wav[:,k]
    p.title="Moving wave"
    push_notebook()
    
show(p)

/Users/mikel/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/mikel/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


In [21]:
interact(update_wav,k=(0,3*t_len-1))

<function __main__.update_wav>

# Summary

Here are all the animated waves, put together with controls for a convenient demo. 


In [22]:
interact(update_exp,k=IntSlider(min=0,max=t_len-1,description='Finite Diff, Explicit, init position'))
interact(update_expD,k=IntSlider(min=0,max=3*t_len-1,description='Finite Diff, Explicit, init velocity'))
interact(update_imp,k=IntSlider(min=0,max=t_len-1,description='Finite Diff, Implicit, init position'))
interact(update_impD,k=IntSlider(min=0,max=3*t_len-1,description='Finite Diff, Implicit, init velocity'))
interact(update_dal,k=IntSlider(min=0,max=t_len-1,description="d'Alembert, init position"))
interact(update_dalD,k=IntSlider(min=0,max=3*t_len-1,description="d'Alembert, init velocity"))
interact(update_comp,k=IntSlider(min=0,max=t_len-1,description='Comparison, exact minus F.D.'))
interact(update_wav,k=IntSlider(min=0,max=3*t_len-1,description='Moving wave (exact)'))


<function __main__.update_wav>

In [23]:
show(p)